# 전국의 치킨집 정보 표시 : 부어치킨

# module import
requests, selenium, BeautifulSoup : 크롤링 모듈
folium : 지도 모듈

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import requests
import folium
from bs4 import BeautifulSoup as bs
import time 
import json

# 테스트 코드 작성(1페이지)

In [2]:
# 부어치킨 매장찾기 페이지
url = 'http://www.boor.co.kr/store/store.aspx' 
store_type = '%ub9e4%uc7a5%ucc3e%uae30'
requests.utils.unquote(store_type)
params = {
    'store_type': store_type
}
resp = requests.get(url, params=params)
soup = bs(resp.text, 'html.parser')
jijum_list = list(map(lambda i:i.text, soup.select('li.sto_title')))
addr_list = list(map(lambda i:i['addr'], soup.select('li.add_text[groupname=Address]')))
tel_list = list(map(lambda i:i.text, soup.select('li.add_text[groupname=Tel]')))
store_list = [(j, a, t) for j, a, t in zip(jijum_list, tel_list, addr_list )]

# 모든 페이지의 정보 크롤링
## 정보
### 대상 url
페이지가 바뀔 때 get방식으로 페이지가 드러나지 않음
-> requests를 사용할 수 없으므로 selenium으로 대체
### selenium
selenium의 headless 모드를 사용하면 크롤링을 제대로 못하는 현상 발생
해결방법을 찾지 못하여 headless 모드 없이 창 켜진채로 크롤링 수행

In [9]:
# # # headless options
# # options = webdriver.ChromeOptions()
# # options.add_argument('headless')
# # options.add_argument('window-size=1920x1080')
# # options.add_argument('disable-gpu')

# chrome_driver = 'D:/bigdata/Python_workspace/jupyter/chromedriver.exe'
# # # headless driver
# # driver = webdriver.Chrome(chrome_driver, chrome_options=options)
# driver = webdriver.Chrome(chrome_driver)
# url = 'http://www.boor.co.kr/store/store.aspx?store_type=%ub9e4%uc7a5%ucc3e%uae30'
# # 1페이지 불러오기
# driver.get(url)
# # 10페이지 단위를 세는 카운터
# cnt = 0
# # 가게 정보를 담을 리스트
# store_list = []
# while True:
#     # 콘솔창에 크롤링 진행상황 표시(*1페이지)
#     print(cnt*10+1,'page')
#     # 페이지를 BeautifulSoup으로 변환
#     resp = driver.page_source
#     soup = bs(resp, 'html.parser')
#     # 데이터 가공 : jijum, tel, addr
#     jijum_list = list(map(lambda i:i.text, soup.select('li.sto_title')))
#     addr_list = list(map(lambda i:i['addr'], soup.select('li.add_text[groupname=Address]')))
#     tel_list = list(map(lambda i:i.text, soup.select('li.add_text[groupname=Tel]')))
#     # 가공한 데이터를 store_list에 저장
#     store_list.extend([{'jijum': j, 'tel': a, 'addr': t} for j, a, t in zip(jijum_list, tel_list, addr_list )])
#     # 크롤링이 제대로 수행되도록 간격 설정
#     time.sleep(1)
#     for i in range(2,11):
#         # 콘솔창에 크롤링 진행상황 표시(*2~*0 페이지)
#         print(cnt*10+i,'page')
#         # 진행하는 페이지가 있을 경우
#         try:
#             elem = driver.find_element_by_css_selector(f'div.PagingCell[onclick="JavaScript:Page({cnt*10 + i})"]')
#             elem.click()
#         # 진행하려는 페이지가 없을 경우 나오도록 조건 설정        
#         except Exception:
#             break
#         # 페이지를 BeautifulSoup으로 변환
#         resp = driver.page_source
#         soup = bs(resp, 'html.parser')
#         # 데이터 가공 : jijum, tel, addr
#         jijum_list = list(map(lambda i:i.text.split('[')[1].split(']')[0], soup.select('li.sto_title')))
#         addr_list = list(map(lambda i:i['addr'], soup.select('li.add_text[groupname=Address]')))
#         tel_list = list(map(lambda i:i.text, soup.select('li.add_text[groupname=Tel]')))
#         # 가공한 데이터를 store_list에 저장
#         store_list.extend([{'jijum': j, 'tel': a, 'addr': t} for j, a, t in zip(jijum_list, tel_list, addr_list )])
#         time.sleep(0.1)
#     # 진행하는 페이지가 있을 경우
#     try:
#         elem = driver.find_element_by_css_selector('a.pageList[title="다음 10개"]')
#         elem.click()
#     # 진행하려는 페이지가 없을 경우 나오도록 조건 설정 
#     except Exception:
#         print('end')
#         break
#     # 페이지를 10페이지 단위로 추가
#     cnt += 1
# driver.close()

In [10]:
def getLatLng(addr):
#     addr = '서울특별시 구로구 디지털로 306'
    app_key = 'KakaoAK' + ' 78106cafe4d656639f171b177dda611f'
    url = 'https://dapi.kakao.com/v2/local/search/address.json'
    params = {
        'query': addr,
        'page': '',
        'AddressSize': ''
    }
    headers  = {
        'Authorization': app_key
    }
    resp = requests.get(url, params=params, headers=headers)
    if resp.json()['documents'] != []:
        json = resp.json()['documents'][0]
        return (json['x'], json['y'])
    else:
        fixed_addr = addr.replace(addr.split(' ')[-1], '').strip()
        if fixed_addr == '':
            print('불러오지 못했습니다 : ', addr)
            return (0, 0)
        else:
            return getLatLng(fixed_addr)
        
def get_avg_point(x_list, y_list):
    x = sum(x_list) / len(x_list)
    y = sum(y_list) / len(y_list)
    return x, y

def draw_map(addr):
    map_noman = folium.Map(location=get_avg_point(addr_point_list),zoom_start=7)
    disabled_sup = list(set(df_noman['장애인편의발급기형태']))
    for x, y in addr_point_list:
        if df_noman['장애인편의발급기형태'][cnt] == disabled_sup[0]:
            add_marker(map_noman, x, y)
    return map_noman

def draw_marker(x, y, tel):
    marker = folium.Marker((x, y), popup=tel)
    icon=folium.Icon(color='orange',icon='glyphicon glyphicon-cutlery')
    icon.add_to(marker)
    return marker

In [4]:
for store in store_list:
    point = getLatLng(store['addr'])
    store['x'] = point[1]
    store['y'] = point[0]

NameError: name 'getLatLng' is not defined

In [132]:
with open('boorchicken.json','w',encoding='utf-8') as f:
    store_list_dump = json.dumps(store_list, indent=4, sort_keys=True, ensure_ascii=False)
    f.write(store_list_dump)

In [9]:
map_boor

NameError: name 'map_boor' is not defined

# requests 모듈 활용 크롤링

In [32]:
import requests
import folium
from bs4 import BeautifulSoup as bs
import time 
import json
from tinydb import TinyDB, Query

In [26]:
url = 'http://www.boor.co.kr/store/store.aspx'
cnt = 0
store_list = []
while True:
    cnt += 1
    params = {
        'page': cnt
    }
    resp = requests.get(url,params=params)
    resp.status_code
    soup = bs(resp.text)
    jijum_list = list(map(lambda i:i.text, soup.select('li.sto_title')))
    if jijum_list == []:
        break
    addr_list = list(map(lambda i:i['addr'], soup.select('li.add_text[groupname=Address]')))
    tel_list = list(map(lambda i:i.text, soup.select('li.add_text[groupname=Tel]')))
    x_list = list(map(lambda i:i['lati'], soup.select('ul.sto_text_box>li')))
    y_list = list(map(lambda i:i['longi'], soup.select('ul.sto_text_box>li')))
    store_zip = zip(jijum_list, tel_list, addr_list, x_list, y_list)
    store_list.extend([{'jijum': j, 'tel': a, 'addr': t, 'x': x, 'y': y} for j, a, t, x, y in store_zip])
for store in store_list:
    if store['x'] == '' or store['y'] == '':
        store['y'], store['x'] = getLatLng(store['addr'])      
store_list

[{'jijum': '부어치킨[가능점]',
  'tel': '031-877-9934',
  'addr': '경기도 의정부시 가능동 732-21',
  'x': '37.7433970807',
  'y': '127.035621563'},
 {'jijum': '부어치킨[가좌1동점]',
  'tel': '032-571-8417',
  'addr': '인천광역시 서구 가좌동 142-36 부어치킨',
  'x': '37.4960848492',
  'y': '126.673722273'},
 {'jijum': '부어치킨[가좌4동점]',
  'tel': '032-583-6699',
  'addr': '인천광역시 서구 가좌동 350-5',
  'x': '37.4871429793',
  'y': '126.685570175'},
 {'jijum': '부어치킨[가좌동점]',
  'tel': '032-576-0709',
  'addr': '인천광역시 서구 가좌동 30-25',
  'x': '37.4939085931',
  'y': '126.684546709'},
 {'jijum': '부어치킨[간석1동점]',
  'tel': '032-425-2727',
  'addr': '인천광역시 남동구 간석1동 285-7 오성상가 104호 부어치킨',
  'x': '37.4605487512',
  'y': '126.707998161'},
 {'jijum': '부어치킨[갈마점_대전지사]',
  'tel': '042-525-9225',
  'addr': '대전광역시 서구 갈마동 351-2 부어치킨',
  'x': '36.3472713049',
  'y': '127.37171823'},
 {'jijum': '부어치킨[갈현2호점]',
  'tel': '02-388-3102',
  'addr': '서울특별시 은평구 갈현동 527-15 부어치킨',
  'x': '37.6146903324',
  'y': '126.909960389'},
 {'jijum': '부어치킨[갈현점]',
  'tel': '02-386-8

In [28]:
x_list = list(map(lambda store:float(store['x']),store_list))
y_list = list(map(lambda store:float(store['y']),store_list))
tel_list = list(map(lambda store:store['tel'],store_list))
map_boor = folium.Map(location=get_avg_point(x_list,y_list), zoom_start=7)
for x, y, tel in zip(x_list,y_list, tel_list):
    draw_marker(x, y, tel).add_to(map_boor)
map_boor

In [33]:
db = TinyDB('boorchicken')

In [36]:
for store in store_list:
    db.insert(store)
db.all()

[{'jijum': '부어치킨[가능점]',
  'tel': '031-877-9934',
  'addr': '경기도 의정부시 가능동 732-21',
  'x': '37.7433970807',
  'y': '127.035621563'},
 {'jijum': '부어치킨[가좌1동점]',
  'tel': '032-571-8417',
  'addr': '인천광역시 서구 가좌동 142-36 부어치킨',
  'x': '37.4960848492',
  'y': '126.673722273'},
 {'jijum': '부어치킨[가좌4동점]',
  'tel': '032-583-6699',
  'addr': '인천광역시 서구 가좌동 350-5',
  'x': '37.4871429793',
  'y': '126.685570175'},
 {'jijum': '부어치킨[가좌동점]',
  'tel': '032-576-0709',
  'addr': '인천광역시 서구 가좌동 30-25',
  'x': '37.4939085931',
  'y': '126.684546709'},
 {'jijum': '부어치킨[간석1동점]',
  'tel': '032-425-2727',
  'addr': '인천광역시 남동구 간석1동 285-7 오성상가 104호 부어치킨',
  'x': '37.4605487512',
  'y': '126.707998161'},
 {'jijum': '부어치킨[갈마점_대전지사]',
  'tel': '042-525-9225',
  'addr': '대전광역시 서구 갈마동 351-2 부어치킨',
  'x': '36.3472713049',
  'y': '127.37171823'},
 {'jijum': '부어치킨[갈현2호점]',
  'tel': '02-388-3102',
  'addr': '서울특별시 은평구 갈현동 527-15 부어치킨',
  'x': '37.6146903324',
  'y': '126.909960389'},
 {'jijum': '부어치킨[갈현점]',
  'tel': '02-386-8